<a href="https://colab.research.google.com/github/sushant-97/Chaabi_Assignment_CSV_Chat/blob/main/CHAABI_Assignment_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers qdrant_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers

In [ ]:
!python --version

Python 3.10.12


In [ ]:
import pandas as pd
import numpy as np
# from sentence_transforxmers import SentenceTransformer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from qdrant_client import QdrantClient

# Load your CSV file into a pandas DataFrame
csv_file_path = '/content/drive/MyDrive/IITG/PLACEMENTS/Assignments/CHAABI/bigBasketProducts.csv'
df = pd.read_csv(csv_file_path)
df = df[:100]

# Inintal Trial

In [ ]:
type(df.iloc[3])

pandas.core.series.Series

In [ ]:
df.iloc[3].to_dict()

{'index': 4,
 'product': 'Cereal Flip Lid Container/Storage Jar - Assorted Colour',
 'category': 'Cleaning & Household',
 'sub_category': 'Bins & Bathroom Ware',
 'brand': 'Nakoda',
 'sale_price': 149.0,
 'market_price': 176.0,
 'type': 'Laundry, Storage Baskets',
 'rating': 3.7,
 'description': 'Multipurpose container with an attractive design and made from food-grade plastic for your hygiene and safety ideal for storing pulses. Grains, spices, and more with easy opening and closing flip-open lid. Strong, durable and transparent body for longevity and easy identification of contents. Multipurpose storage solution for your daily needs stores your everyday food essentials in style with the Nakoda container set. With transparent bodies, you can easily identify your stored items without having to open the lids. These containers are ideal for storing a large variety of items such as food grains, snacks and pulses to sugar, spices, condiments and more. Featuring unique flip-open lids, you c

In [ ]:
# Load a pre-trained model for generating embeddings
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Generate embeddings for each row in the DataFrame
embeddings = model.encode(df['description'].astype(str), convert_to_tensor=False)

# Convert embeddings to PyTorch tensors
embeddings = torch.tensor(embeddings, dtype=torch.float32)

NameError: ignored

In [ ]:
# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_size, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Set the input and encoding dimensions
input_dim = len(embeddings[0])
encoding_dim = 200

# Create the autoencoder model
autoencoder = Autoencoder(input_dim, encoding_dim)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# Train the autoencoder
num_epochs = 50
for epoch in range(num_epochs):
    outputs = autoencoder(embeddings)
    loss = criterion(outputs, embeddings)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Get the reduced embeddings
with torch.no_grad():
    reduced_embeddings = autoencoder.encoder(embeddings).numpy()

# Convert the reduced embeddings to a list of lists
embedding_list = reduced_embeddings.tolist()


Epoch [10/50], Loss: 0.2139
Epoch [20/50], Loss: 0.0728
Epoch [30/50], Loss: 0.0554
Epoch [40/50], Loss: 0.0536
Epoch [50/50], Loss: 0.0528


In [ ]:
len(embedding_list[0])

200

In [ ]:
from qdrant_client import models, QdrantClient

In [ ]:
qdrant = QdrantClient(":memory:")

In [ ]:
vectors_=models.VectorParams(
        size=200,  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    )

In [ ]:
qdrant.recreate_collection(
    collection_name="test_collection",
    vectors_config=vectors_,
)

True

In [ ]:
# records = [x for idx, doc in enumerate(embedding_list)]

In [ ]:
# records=[models.Record(id=idx, vector=encoder.encode(doc["descri ption"]).tolist(), payload=doc) for idx, doc in enumerate(documents)]

In [ ]:
# qdrant.upload_records(
#     collection_name="test_collection",
#     records=[
#         models.Record(
#             id=idx, vector=encoder.encode(doc["description"]).tolist(), payload=doc
#         )
#         for idx, doc in enumerate(documents)
#     ],
# )

In [ ]:
from qdrant_client.http.models import PointStruct

vec = [PointStruct(id = idx, vector = x, payload = df.iloc[idx]) for idx,x in enumerate(embedding_list)]

operation_info = qdrant.upsert(
    collection_name="test_collection",
    wait=True,
    points=vec,
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [ ]:
search_result = qdrant.search(
    collection_name="test_collection", query_vector=embedding_list[4], limit=3
)


In [ ]:
len(search_result)

3

In [ ]:
def print_result(search_result):
  # Print information for each scored point
  for point in search_result:
      print(f"Product ID: {point.id}")
      print(f"Score: {point.score}")

      # Print payload details
      payload = point.payload
      print(f"Product Name: {payload['product']}")
      print(f"Category: {payload['category']}")
      print(f"Sub-category: {payload['sub_category']}")
      print(f"Brand: {payload['brand']}")
      print(f"Sale Price: {payload['sale_price']}")
      print(f"Market Price: {payload['market_price']}")
      print(f"Type: {payload['type']}")
      print(f"Rating: {payload['rating']}")
      print(f"Description: {payload['description']}")

      print("\n----------------------\n")


In [ ]:
df.head()

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [ ]:
embedding_list[0][0]

1.1945823431015015

In [ ]:
# Query
query = "How many products are of type Hair Oil and Serum?"
query_vec_model = model.encode(query, convert_to_tensor=False)
# Convert embeddings to PyTorch tensors
query_vec_model = torch.tensor(query_vec_model, dtype=torch.float32)

# Get the reduced embeddings
with torch.no_grad():
    reduced_embeddings = autoencoder.encoder(query_vec_model).numpy()

# Convert the reduced embeddings to a list of lists
query_vector_ = reduced_embeddings.tolist()

In [ ]:
query_vector_

[1.3099710941314697,
 0.7554594874382019,
 0.35983678698539734,
 0.8897739052772522,
 1.092047095298767,
 0.48170894384384155,
 0.8669180870056152,
 1.371763825416565,
 1.3771411180496216,
 0.9507513046264648,
 1.329007863998413,
 1.3444126844406128,
 1.04909086227417,
 1.2602484226226807,
 1.4953609704971313,
 1.4163143634796143,
 1.5264003276824951,
 1.258849024772644,
 0.9477965235710144,
 0.808463454246521,
 1.435517430305481,
 1.3615449666976929,
 0.9383190274238586,
 1.2793042659759521,
 1.26472806930542,
 1.3350491523742676,
 0.0,
 0.9604185223579407,
 1.314059853553772,
 1.3818464279174805,
 1.5091183185577393,
 1.0339182615280151,
 1.3749394416809082,
 1.2502436637878418,
 1.0954664945602417,
 0.9696476459503174,
 0.0,
 1.2742242813110352,
 0.7824317812919617,
 0.8719455599784851,
 1.565778374671936,
 1.1682097911834717,
 0.0,
 1.4655669927597046,
 1.3645809888839722,
 0.8503220677375793,
 1.3474467992782593,
 0.9558589458465576,
 1.3721716403961182,
 0.41091814637184143,
 0.0

In [ ]:
search_result = qdrant.search(
    collection_name="test_collection", query_vector=query_vector_, limit=3
)

In [ ]:
print_result(search_result)

Product ID: 16
Score: 0.987340532957319
Product Name: Smooth Skin Oil - For Dry Skin
Category: Beauty & Hygiene
Sub-category: Skin Care
Brand: Aroma Treasures
Sale Price: 324.0
Market Price: 360.0
Type: Aromatherapy
Rating: 5.0
Description: Specially crafted for dry skin, this richly formulated skin oil restores the natural moisture of the skin, improves its health and youthfulness. Not to mention how soft, shiny and smooth it makes your skin.


Aroma Treasures, pioneers of aromatherapy have put in years of research and study to give you the ultimate range of beauty care products.

----------------------

Product ID: 57
Score: 0.9869918009899178
Product Name: Argan-Liquid Gold Hair Spa
Category: Beauty & Hygiene
Sub-category: Hair Care
Brand: Aroma Treasures
Sale Price: 199.5
Market Price: 210.0
Type: Hair & Scalp Treatment
Rating: 4.4
Description: Our beautifully crafted Hair Spa Collection promises to be the perfect solution for nourishing and rejuvenating your tresses. Made with Arg

In [ ]:
search_result1 = qdrant.search(
    collection_name="test_collection", query_vector=embedding_list[4], limit=3
)

print_result(search_result1)
print("***************")

search_result2 = qdrant.search(
    collection_name="test_collection", query_vector=embedding_list[42], limit=3
)
print_result(search_result2)

Product ID: 4
Score: 1.0000000055017462
Product Name: Creme Soft Soap - For Hands & Body
Category: Beauty & Hygiene
Sub-category: Bath & Hand Wash
Brand: Nivea
Sale Price: 162.0
Market Price: 162.0
Type: Bathing Bars & Soaps
Rating: 4.4
Description: Nivea Creme Soft Soap gives your skin the best care that it must get. The soft bar consists of Vitamins F and Almonds which are really skin gracious and help you get great skin. It provides the skin with moisture and leaves behind flawless and smooth skin. It makes sure that your body is totally free of germs & dirt and at the same time well nourished.For Beauty tips, tricks & more visit https://bigbasket.blog/

----------------------

Product ID: 42
Score: 0.9936772142141496
Product Name: Flavoured Cream Wafer Roll - Strawberry
Category: Gourmet & World Food
Sub-category: Chocolates & Biscuits
Brand: Twister
Sale Price: 275.0
Market Price: 275.0
Type: Cookies, Biscotti, Wafer
Rating: 4.3
Description: Very crispiest texture and pleasant odo

In [ ]:
# Get user input from the command line
user_input = input("Enter something: ")

# Print the user input
print("You entered:", user_input)

Enter something: how are u?
You entered: how are u?


# Final Run

In [ ]:
#clone github repo
!git clone "https://github.com/sushant-97/Chaabi_Assignment_CSV_Chat.git"

Cloning into 'Chaabi_Assignment_CSV_Chat'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 17 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (17/17), 5.10 KiB | 5.10 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [ ]:
import os
os.chdir("/content/Chaabi_Assignment_CSV_Chat/COLAB")

In [ ]:
!python all.py

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>
tokenizer_config.json: 100% 287/287 [00:00<00:00, 1.26MB/s]
tokenizer.json: 100% 2.73M/2.73M [00:00<00:00, 61.7MB/s]
special_tokens_map.json: 100% 281/281 [00:00<00:00, 1.62MB/s]
config.json: 100% 1.05k/1.05k [00:00<00:00, 4.17MB/s]
configuration_falcon.py: 100% 7.16k/7.16k [00:00<00:00, 27.8MB/s]
A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py: 100% 56.9k/56.9k [00:00<00:00, 117MB/s]
A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
pytorch_model.bin.index.json: 100% 1

In [ ]:
%pip install transformers bitsandbytes accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 25.6 MB/s eta 0:00:00


In [ ]:
!python main.py

Enter your query: list down all hair products
Traceback (most recent call last):
  File "/content/Chaabi_Assignment_CSV_Chat/COLAB/main.py", line 36, in <module>
    context_list = qdrant_client.search(
  File "/usr/local/lib/python3.10/dist-packages/qdrant_client/qdrant_client.py", line 324, in search
    return self._client.search(
  File "/usr/local/lib/python3.10/dist-packages/qdrant_client/local/qdrant_local.py", line 162, in search
    collection = self._get_collection(collection_name)
  File "/usr/local/lib/python3.10/dist-packages/qdrant_client/local/qdrant_local.py", line 121, in _get_collection
    raise ValueError(f"Collection {collection_name} not found")
ValueError: Collection test_collection not found
